In [1]:
import Acqu as aq
import Timepix
import numpy as np
import ROOT
from rootpy.plotting import Hist, Hist2D, histogram, Canvas
from ipykernel import kernelapp as app

Welcome to JupyROOT 6.16/00


In [2]:
inFile = '/w/work0/mainz/2019_05_Timepix3-Acqu/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


In [3]:
TimepixAData = []
TimepixBData = []

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData.append(Timepix.Decode(nHitsA,encodedA))
        TimepixBData.append(Timepix.Decode(nHitsB,encodedB))       
aq.runFunction(plotTimepix,0,50000)

In [4]:
ToAAData = []
ToAA = []

for i in range(len(TimepixAData)):
    ToAAData.append(TimepixAData[0]['ToA'])
for j in range(len(ToAAData)):
    for k in range(len(ToAAData[j])):
        ToAA.append(ToAAData[j][k])

ToAA.sort()
        
ToABData = []
ToAB = []

for i in range(len(TimepixBData)):
    ToABData.append(TimepixBData[0]['ToA'])
for j in range(len(ToABData)):
    for k in range(len(ToABData[j])):
        ToAB.append(ToABData[j][k])

ToAB.sort()

print(ToAA)

[12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 12700, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37385, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386, 37386

In [5]:
cl = [None]*100
cn = 0
blen = 0
b = 0

blen = 100 #readTimepixBufferFromFile()